# Fuel Weight

This section describes the code used for computing the fuel weight fraction for a given MTOW value and [parameter class](parameters.ipynb) object. As described in [mission analysis section](../performance/mission_analysis.ipynb), only the methods for cruise and loiter segments are refined. The remaining segments are same as initial weight estimation. The cruise and loiter segments are divided into many smaller parts and compute fuel burnt in those smaller segments. The total total fuel burnt is just the sum of all the fuel burnt in those smaller segments.

Below code block defines function for computing cruise and loiter weight fraction.

In [7]:
import numpy as np
import nbimporter
from parameters import AircraftParameters

def compute_cruise_weight_fraction(parameters: AircraftParameters, weight, R, cruise_speed, num_segements=100):
    """
        Compute the weight fraction for cruise segment

        Parameters
        ----------
        weight : float
            Initial weight at the beginning of cruise (lbs)
        R : float
            Range of cruise (ft)
        cruise_speed : float
            speed during cruise (ft/s)
        num_segements : int
            Number of segments to divide the cruise into
        
        Returns
        -------
        cruise_weight_fraction: float
            Total weight fraction for the cruise segment
        avg_power_req: float
            Average power required during cruise (hp)
        avg_CL_cruise: float
            Average lift coefficient during cruise
    """

    # Parameters
    A = parameters.A
    S = parameters.S
    rho = parameters.rho_cruise
    CD0 = parameters.CD0
    e = parameters.e
    PI = parameters.PI
    Cbhp = parameters.Cbhp
    install_loss_factor = parameters.install_loss_factor
    prop_eff_cruise = parameters.prop_eff_cruise

    segment_range = R / num_segements # segment range, ft
    segment_time = segment_range / cruise_speed / 3600 # hr
    q = 0.5 * rho * cruise_speed**2 # lbs/sq ft
    K = 1/PI/A/e

    # initialize empty lists
    cruise_fuel = 0
    power_req = []
    cruise_CL = []

    for i in range(num_segements):

        # Compute Cd
        CL = weight / q / S
        cruise_CL.append(CL)
        CD = CD0 + K * CL**2

        # Compute power required for this segment
        T = CD * q * S / install_loss_factor # lbs
        segment_power = T * cruise_speed / prop_eff_cruise / 550 # hp
        power_req.append(segment_power)
        
        # fuel spent 
        segment_fuel = Cbhp * segment_power * segment_time # lbs
        cruise_fuel += segment_fuel

        weight = weight - segment_fuel # weight at the end of the segment

    cruise_weight_fraction  = weight / (weight + cruise_fuel)
    avg_power_req = np.mean(power_req)
    avg_cruise_CL = np.mean(cruise_CL)

    return cruise_weight_fraction, avg_power_req, avg_cruise_CL

def compute_loiter_weight_fraction(parameters: AircraftParameters, weight, E, num_segements=100):
    """
        Compute the weight fraction for loiter segment

        Parameters
        ----------
        weight : float
            Initial weight at the beginning of loiter (lbs)
        E : float
            loiter time (sec)
        num_segements : int
            Number of segments to divide the loiter into
        
        Returns
        -------
        weight_fraction: float
            Total weight fraction for the loiter segment
        avg_power_req: float
            Average power required during loiter (hp)
        avg_speed_loiter: float
            Average speed during loiter
    """

    # Parameters
    A = parameters.A
    S = parameters.S
    rho = parameters.rho_loiter
    CD0 = parameters.CD0
    e = parameters.e
    PI = parameters.PI
    Cbhp = parameters.Cbhp
    install_loss_factor = parameters.install_loss_factor
    prop_eff_loiter = parameters.prop_eff_loiter

    segment_time = E / num_segements / 3600 # hr

    K = 1/PI/A/e
    CL = (3*CD0/K)**0.5 # for max endurance for a prop airplane
    CD = CD0 + K * CL**2

    # initialize empty lists
    loiter_fuel = 0
    power_req = []
    loiter_speed = []

    for i in range(num_segements):

        V = ( weight/S * 2/rho * 1/CL )**0.5 # ft/s
        loiter_speed.append(V)
        q = 0.5 * rho * V**2 # lbs/sq ft

        # Compute power required for this segment
        T = CD * q * S / install_loss_factor # lbs
        segment_power = T * V / prop_eff_loiter / 550 # hp
        power_req.append(segment_power)

        # fuel spent 
        segment_fuel = Cbhp * segment_power * segment_time # lbs
        loiter_fuel += segment_fuel

        weight = weight - segment_fuel # weight at the end of the segment

    loiter_weight_fraction  = weight / (weight + loiter_fuel)
    avg_power_req = np.mean(power_req)
    avg_loiter_speed = np.mean(loiter_speed)
    
    return loiter_weight_fraction, avg_power_req, avg_loiter_speed

Below code block defines a function for fuel weight estimation:

In [8]:
def compute_fuel_weight(parameters: AircraftParameters, weight):
        """
            Method to compute fuel weight fraction for a given maximum takeoff weight

            This function essentially performs mission analysis
        """

        R = parameters.range
        W0 = weight
        W1_W0 = parameters.W1_W0
        W2_W1 = parameters.W2_W1
        W4_W3 = parameters.W4_W3

        # Compute cruise weight fraction
        weight = W2_W1 * W1_W0 * weight # weight at the start of cruise
        W3_W2, avg_cruise_power, avg_cruise_CL = compute_cruise_weight_fraction(parameters, weight, R, parameters.cruise_speed)
        parameters.W3_W2 = W3_W2
        parameters.avg_cruise_power = avg_cruise_power
        parameters.avg_cruise_CL = avg_cruise_CL

        # Compute loiter weight fraction
        W5_W4 = parameters.W5_W4
        weight = W5_W4 * W4_W3 * W3_W2 * weight # weight at the start of loiter
        W6_W5, avg_loiter_power, avg_loiter_speed = compute_loiter_weight_fraction(parameters, weight, parameters.endurance)
        parameters.W6_W5 = W6_W5
        parameters.avg_loiter_power = avg_loiter_power
        parameters.avg_loiter_speed = avg_loiter_speed

        W7_W6 = parameters.W7_W6
        W8_W7 = parameters.W8_W7

        # compute final weight fraction
        W8_W0 = W8_W7 * W7_W6 * W6_W5 * W5_W4 * W4_W3 * W3_W2 * W2_W1 * W1_W0 # mission weight fraction
        Wf_W0 = 1.06 * (1 - W8_W0) # 6% is for trapped unusable fuel

        parameters.fuel_weight = Wf_W0 * W0

        return parameters.fuel_weight # fuel weight

Below code block demonstrates how the fuel weight function can be used:

In [9]:
# Parameters
A = 8
S = 134 # sq ft
max_power = 595 # hp
aircraft = AircraftParameters(A, S, max_power)

# Weight
MTOW = 5000 # lbs
fuel_weight = compute_fuel_weight(aircraft, MTOW) # lbs

print(f"Fuel weight for MTOW of {MTOW:.0f} lbs: {fuel_weight:.0f} lbs")

Fuel weight for MTOW of 5000 lbs: 1538 lbs


This concludes the fuel weight fraction estimation module, next section describes module for empty weight estimation.